
# Hotel Bookings Dataset — End‑to‑End Analysis

This notebook is part of my data analytics portfolio. It demonstrates an end‑to‑end workflow on the popular **Hotel Bookings** dataset: data loading, cleaning, feature engineering, exploratory analysis, visualization, and insights.

> Tools: **Python**, **pandas**, **matplotlib**.



## Business Task (Ask)
Identify factors correlated with **booking cancellations** and surface actionable insights to **reduce cancellations** and **optimize revenue (ADR)**.


In [ ]:

# --- Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Optional: display settings
pd.set_option('display.max_columns', 50)

# Data path (place the CSV in the repository at: data/hotel_bookings.csv)
DATA_PATH = 'data/hotel_bookings.csv'


## Prepare: Load & Preview Data

In [ ]:

# Load CSV (download the "hotel_bookings.csv" dataset and place it in data/)
df = pd.read_csv(DATA_PATH)

# Basic shape and preview
print('Shape:', df.shape)
df.head()



## Process: Clean & Engineer Features
Steps:
- Standardize column names (already lowercase in the public dataset).
- Handle missing values (children, country, agent/company).
- Create **total_nights** and **total_guests**.
- Create a proper **arrival_month_num** to sort months chronologically.


In [ ]:

# Copy to avoid modifying original
data = df.copy()

# Replace obvious missing numeric fields with 0 and strings with 'Unknown'
for col in ['children', 'babies', 'agent', 'company']:
    if col in data.columns:
        if data[col].dtype.kind in 'biufc':
            data[col] = data[col].fillna(0)
        else:
            data[col] = data[col].fillna('Unknown')

# Some datasets have children as floats with NaNs; ensure numeric
if 'children' in data.columns:
    data['children'] = pd.to_numeric(data['children'], errors='coerce').fillna(0).astype(int)

# Total nights and guests
if {'stays_in_weekend_nights','stays_in_week_nights'}.issubset(data.columns):
    data['total_nights'] = data['stays_in_weekend_nights'] + data['stays_in_week_nights']
else:
    data['total_nights'] = np.nan

guest_cols = [c for c in ['adults','children','babies'] if c in data.columns]
if guest_cols:
    data['total_guests'] = data[guest_cols].sum(axis=1)
else:
    data['total_guests'] = np.nan

# Month number for chronological sorting
if 'arrival_date_month' in data.columns:
    month_order = ['January','February','March','April','May','June','July','August','September','October','November','December']
    month_map = {m:i+1 for i,m in enumerate(month_order)}
    data['arrival_month_num'] = data['arrival_date_month'].map(month_map).astype('Int64')

# Convert reservation_status_date to datetime if present
for dtcol in ['reservation_status_date']:
    if dtcol in data.columns:
        data[dtcol] = pd.to_datetime(data[dtcol], errors='coerce')

# Drop duplicates
data = data.drop_duplicates()
data.head()


## Analyze: Key Metrics

In [ ]:

# Overall cancellation rate
if 'is_canceled' in data.columns:
    cancel_rate = data['is_canceled'].mean()
    print(f'Overall cancellation rate: {cancel_rate:.2%}')

# ADR summary
if 'adr' in data.columns:
    print('\nADR (Average Daily Rate) summary:')
    print(data['adr'].describe())

# Cancellation by market segment
group_cols = [c for c in ['market_segment','deposit_type','customer_type'] if c in data.columns]
summaries = {}
for col in group_cols:
    summaries[col] = (data.groupby(col)['is_canceled'].mean().sort_values(ascending=False) if 'is_canceled' in data.columns else None)

summaries



## Share: Visualizations
The following figures will be saved under `images/` when you run the notebook locally.


In [ ]:

# Create images directory if not exists
import os
os.makedirs('images', exist_ok=True)

# 1) Cancellation rate by deposit type
if {'deposit_type','is_canceled'}.issubset(data.columns):
    fig = plt.figure(figsize=(8,5))
    (data.groupby('deposit_type')['is_canceled'].mean()
         .sort_values(ascending=False)
         .plot(kind='bar'))
    plt.title('Cancellation Rate by Deposit Type')
    plt.ylabel('Cancellation Rate')
    plt.xlabel('Deposit Type')
    plt.tight_layout()
    plt.savefig('images/cancel_by_deposit.png', dpi=200)
    plt.show()

# 2) Cancellation rate by market segment
if {'market_segment','is_canceled'}.issubset(data.columns):
    fig = plt.figure(figsize=(8,5))
    (data.groupby('market_segment')['is_canceled'].mean()
         .sort_values(ascending=False)
         .plot(kind='bar'))
    plt.title('Cancellation Rate by Market Segment')
    plt.ylabel('Cancellation Rate')
    plt.xlabel('Market Segment')
    plt.tight_layout()
    plt.savefig('images/cancel_by_segment.png', dpi=200)
    plt.show()

# 3) ADR by month (median)
if {'arrival_date_month','adr'}.issubset(data.columns):
    fig = plt.figure(figsize=(9,5))
    (data.groupby(['arrival_date_month','arrival_month_num'])['adr']
         .median()
         .reset_index()
         .sort_values('arrival_month_num')
         .set_index('arrival_date_month')['adr']
         .plot(kind='bar'))
    plt.title('Median ADR by Month')
    plt.ylabel('ADR (Median)')
    plt.xlabel('Arrival Month')
    plt.tight_layout()
    plt.savefig('images/adr_by_month.png', dpi=200)
    plt.show()



## Act: Insights & Recommendations (to edit after running)
- **Deposit type** appears strongly associated with cancellation rate. Consider revisiting deposit policies (e.g., non‑refundable discounts vs. flexible terms).
- **Market segments** with higher cancellations may need clearer communication, pricing transparency, or targeted retention offers.
- **ADR by month** highlights seasonality. Use this to align promotional calendars and staffing.

> Replace these bullets with your findings after executing the notebook on your local machine.
